In [1]:
#dataset = pd.read_csv('mnist_test.csv')

In [2]:
import numpy as np
import pandas as pd

qtInter,qtOut,qtInput,e,alpha,epoca = 100,10,784,0,0.1,0
dataset = pd.read_csv('mnist_train.csv')
v = np.random.randn(qtInput, qtInter) / np.sqrt(qtInput)
w = np.random.randn(qtInter, qtOut) / np.sqrt(qtInter)
ativacao = 'sig'

In [3]:
grafico = np.array([[0]*2])

In [4]:
def trataLinha(inp):
    retorno = np.array(inp)
    return retorno/255

def targetVetor(labels, num_classes=10):
    retTarget = np.eye(num_classes)[labels]
    return retTarget

def inputZ(inp):
    inX = np.zeros(qtInter)
    for j in range (qtInter):
        inX[j] = np.sum(inp * np.array(v[:,j]))
    return inX

def inputY(inp,pesos):
    inX = np.zeros(qtOut)
    for k in range (qtOut):
        inX[k] = np.sum(inp * np.array(pesos[:,k]))
    return inX

def deltainJ (dK,peso):
    inJ = np.zeros(qtInter)
    for j in range (qtInter):
        for k in range (qtOut):
            inJ[j] += dK[k] * peso[j][k]
    return inJ

def funcAtivacao(x,func) :
    if (func == 'relu') : 
        return np.maximum(x,0) 
    elif (func == 'sig') : 
        return (1/(1+np.exp(-x)))
    elif (func == 'tanh'):
        return np.tanh(x)

def derivada(x,func):        
    if (func == 'relu') : 
        return (0 if x<0 else 1)
    elif (func == 'sig') : 
        return (x * (1 - x))
    elif (func == 'tanh') :
        return (1 - np.power(x,2))

def verificaAcerto(tar,out):
    indTar = np.unravel_index(np.argmax(tar, axis=None), tar.shape)
    indOut = np.unravel_index(np.argmax(out, axis=None), out.shape)
    if (indTar == indOut):
        return True
    else :
        return False

def deltaK(targetK,Yk,YinK,func) :
    dK = np.zeros(qtOut)
    for i in range (qtOut) :
        erro = targetK[i] - Yk[i]
        deriv = derivada(YinK[i],func)
        dK[i] = erro * deriv
    return dK

def deltaJ(inJ,inZ,func):
    delJ = np.zeros(qtInter)
    for j in range (qtInter):
        deriv = derivada(inZ[j],func)
        delJ[j] = inJ[j] * deriv
    return delJ

def deltaW(dK,Ze):
    dW = np.empty_like(w)
    for j in range(qtInter):
        for k in range(qtOut):
            dW[j][k] = alpha * dK[k] * Ze[j]
    return dW

def deltaV(linha,dJ):
    dV = np.empty_like(v)
    for i in range(qtInput):
        for j in range (qtInter):
            dV[i][j] = alpha * dJ[j] * linha[i]
    return dV

In [5]:
def algoritmo(isTreino,qtEpoca):
    global epoca
    for i in range (qtEpoca):
        readLine(dataset,isTreino)
        epoca+=1

In [6]:
def readLine(dSet,isTreino):
    global e
    r,e = 1,0
    for row in dSet.itertuples(index=False):
        linha = list(row)
        target = targetVetor(linha.pop(0))
        linha = trataLinha(linha)
        forward(linha,isTreino,target,r)
        r+=1
        
def forward(linha,isTreino,target,r):
    global e
    global grafico
    inZ = inputZ(linha)
    Z = funcAtivacao(inZ,ativacao)
    inY = inputY(Z,w)
    Y = funcAtivacao(inY,ativacao)
    if (not verificaAcerto(target,Y)):
        e+=1
    if isTreino:
        backPropagation(inZ,Z,inY,Y,target,linha)
    a = 100 - ((e/r)*100)
    if (r!=0 and r%20 == 0):
        print (target)
        print (Y)
        print (f'Linha: {r} Epoca: {epoca} Erros: {e}  Acerto: {a} %' + '\n' + '_________' )
        grafico = np.append(grafico,[[r,a]],axis=0)
    
def backPropagation(inZ,Z,inY,Y,target,linha):
    delK,inJ,delJ =[],[],[]
    delK = deltaK(target,Y,Y,ativacao) 
    inJ  = deltainJ(delK,w)
    delJ = deltaJ(inJ,Z,ativacao)
    atualizaPesos(delK,Z,linha,delJ)

In [7]:
def atualizaPesos(delK,Z,linha,delJ):
    global w
    global v
    correcaoW = deltaW(delK,Z)
    correcaoV = deltaV(linha,delJ)
    w = corrigePeso(w,correcaoW)
    v = corrigePeso(v,correcaoV)
    
def corrigePeso(peso,delta):
    return np.add(peso,delta)

In [8]:
algoritmo(True,1)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0.11438398 0.20052867 0.14949408 0.17857493 0.13899523 0.14476267
 0.19599052 0.1419781  0.14192557 0.1231479 ]
Linha: 20 Epoca: 0 Erros: 18  Acerto: 10.0 %
_________
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
[0.12906929 0.1122382  0.1103349  0.11810344 0.10856781 0.10401151
 0.15748263 0.12400515 0.10117125 0.12117707]
Linha: 40 Epoca: 0 Erros: 36  Acerto: 10.0 %
_________
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[0.12646759 0.09575025 0.07641452 0.13778176 0.10781264 0.08270176
 0.08483338 0.11238485 0.10767546 0.18913682]
Linha: 60 Epoca: 0 Erros: 53  Acerto: 11.666666666666671 %
_________
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
[0.15914571 0.24651358 0.0700366  0.08585739 0.10544057 0.08204402
 0.09691022 0.10618966 0.07562445 0.10334308]
Linha: 80 Epoca: 0 Erros: 69  Acerto: 13.75 %
_________
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[0.14135497 0.12473549 0.067575   0.10979777 0.10707702 0.06214505
 0.12056349 0.14992111 0.09685533 0.09737117]
Linha: 100 Epoca: 0 Erros: 88  Acerto: 1

[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0.17272116 0.0686446  0.05110299 0.1531628  0.06589638 0.13256964
 0.12554807 0.15544306 0.23438408 0.10993167]
Linha: 800 Epoca: 0 Erros: 533  Acerto: 33.375 %
_________
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
[0.09464725 0.0342116  0.09489619 0.07855703 0.10681331 0.12222435
 0.09156155 0.46106    0.10963444 0.27686263]
Linha: 820 Epoca: 0 Erros: 541  Acerto: 34.02439024390243 %
_________
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
[0.03405498 0.14416505 0.07038321 0.04287609 0.10574333 0.13195984
 0.05875629 0.54897817 0.09714679 0.27652133]
Linha: 840 Epoca: 0 Erros: 550  Acerto: 34.52380952380952 %
_________
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0.50525813 0.01535671 0.10689382 0.17633646 0.25495623 0.16457495
 0.13086411 0.08143791 0.08416915 0.0459599 ]
Linha: 860 Epoca: 0 Erros: 560  Acerto: 34.883720930232556 %
_________
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0.0692531  0.1283684  0.14351969 0.23571588 0.16543558 0.06807188
 0.09211355 0.08684269 0.10903133 0.06603091]
Linha

[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[0.00709537 0.74625696 0.23673376 0.11368736 0.05558659 0.09115271
 0.03445959 0.07164263 0.0789402  0.06718319]
Linha: 1580 Epoca: 0 Erros: 830  Acerto: 47.46835443037975 %
_________
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
[0.10708913 0.04018102 0.06930067 0.62719075 0.03405552 0.20928659
 0.01798796 0.07760202 0.14486451 0.10405593]
Linha: 1600 Epoca: 0 Erros: 836  Acerto: 47.75 %
_________
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
[0.05717946 0.01615796 0.02923521 0.04254622 0.11017708 0.09276403
 0.0139887  0.81449227 0.04063374 0.39617206]
Linha: 1620 Epoca: 0 Erros: 844  Acerto: 47.90123456790123 %
_________
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
[0.02452472 0.03023218 0.02706037 0.05316874 0.10236158 0.08791821
 0.01209707 0.73179332 0.04741087 0.35668618]
Linha: 1640 Epoca: 0 Erros: 850  Acerto: 48.170731707317074 %
_________
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0.00884965 0.03888942 0.04142133 0.04289035 0.22142636 0.08313089
 0.05107888 0.19302559 0.13420632 0.472983  ]
Li

[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
[0.01563768 0.11474009 0.43792348 0.31450045 0.05855977 0.14241725
 0.00225719 0.02126602 0.2965955  0.03006125]
Linha: 2340 Epoca: 0 Erros: 996  Acerto: 57.43589743589744 %
_________
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
[0.01484673 0.05812489 0.64336163 0.46885943 0.16224783 0.06493916
 0.00765133 0.00522204 0.29542912 0.01565472]
Linha: 2360 Epoca: 0 Erros: 1000  Acerto: 57.6271186440678 %
_________
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
[0.01305435 0.20875723 0.19657079 0.09583023 0.03594514 0.04208532
 0.00601667 0.07709965 0.23001655 0.13484149]
Linha: 2380 Epoca: 0 Erros: 1006  Acerto: 57.73109243697479 %
_________
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[0.02339781 0.0186918  0.87118463 0.03734926 0.08577861 0.02509903
 0.05987469 0.00310815 0.52038201 0.0171596 ]
Linha: 2400 Epoca: 0 Erros: 1007  Acerto: 58.04166666666667 %
_________
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[0.00526001 0.38191523 0.28206237 0.05072596 0.18029074 0.0771247
 0.00686817 0.02083755 0.42095733 0.

[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0.00577923 0.01444757 0.00761151 0.00864417 0.2309734  0.06255306
 0.04605805 0.09278001 0.1710035  0.65626534]
Linha: 3100 Epoca: 0 Erros: 1129  Acerto: 63.58064516129032 %
_________
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0.01990337 0.07607028 0.17959654 0.41542287 0.01983328 0.05891627
 0.01282824 0.01178268 0.18504081 0.00859607]
Linha: 3120 Epoca: 0 Erros: 1131  Acerto: 63.75 %
_________
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
[0.01244806 0.00317723 0.04942078 0.00114392 0.15895473 0.09516161
 0.87773342 0.02280459 0.01689839 0.1234817 ]
Linha: 3140 Epoca: 0 Erros: 1134  Acerto: 63.88535031847134 %
_________
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
[0.02389801 0.01448336 0.07423198 0.76793068 0.01938675 0.12207809
 0.00597201 0.01245379 0.33202495 0.01997022]
Linha: 3160 Epoca: 0 Erros: 1136  Acerto: 64.0506329113924 %
_________
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[0.00901565 0.0216915  0.95819052 0.02650264 0.0349459  0.00594069
 0.09072386 0.0128735  0.06843252 0.01096897]


[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0.20361564 0.00201031 0.00284251 0.08972846 0.0041255  0.66655752
 0.03417682 0.00788982 0.10573293 0.03095241]
Linha: 3840 Epoca: 0 Erros: 1238  Acerto: 67.76041666666666 %
_________
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0.07686339 0.00382606 0.00242583 0.16077911 0.00425917 0.59839132
 0.02990008 0.02105439 0.07560391 0.04222131]
Linha: 3860 Epoca: 0 Erros: 1239  Acerto: 67.90155440414507 %
_________
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[0.0672032  0.00587811 0.13308657 0.02451125 0.0039332  0.32388132
 0.00983744 0.00400428 0.83992256 0.02191347]
Linha: 3880 Epoca: 0 Erros: 1242  Acerto: 67.98969072164948 %
_________
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
[0.01577324 0.01272713 0.00462726 0.00768656 0.01508344 0.06129625
 0.00133884 0.95773605 0.03782965 0.21746668]
Linha: 3900 Epoca: 0 Erros: 1242  Acerto: 68.15384615384616 %
_________
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
[0.00348678 0.01775954 0.06710995 0.00111825 0.08762172 0.05834591
 0.96745269 0.00274937 0.03969796

[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
[3.48630857e-03 5.61679595e-02 5.43586744e-02 4.83076552e-02
 3.27053365e-03 6.24074107e-03 2.20677864e-04 9.39730773e-01
 2.36501998e-02 7.68243258e-02]
Linha: 4560 Epoca: 0 Erros: 1329  Acerto: 70.85526315789474 %
_________
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
[0.05525956 0.00529817 0.01634933 0.74112677 0.00104886 0.27798433
 0.01410902 0.01052753 0.01823314 0.00242154]
Linha: 4580 Epoca: 0 Erros: 1331  Acerto: 70.93886462882097 %
_________
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[0.0451581  0.00118624 0.97936446 0.01599622 0.01060559 0.00131197
 0.03791603 0.00236221 0.05216471 0.00399345]
Linha: 4600 Epoca: 0 Erros: 1333  Acerto: 71.02173913043478 %
_________
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
[0.00178215 0.19909879 0.00605862 0.00076978 0.62998016 0.04650748
 0.0555968  0.02060455 0.007917   0.06937239]
Linha: 4620 Epoca: 0 Erros: 1334  Acerto: 71.12554112554113 %
_________
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[5.79821968e-04 8.21116626e-01 6.62009004e-02 3.16182118e-0

KeyboardInterrupt: 

In [9]:
np.delete(grafico,0,0)

array([[  20.        ,   10.        ],
       [  40.        ,   10.        ],
       [  60.        ,   11.66666667],
       [  80.        ,   13.75      ],
       [ 100.        ,   12.        ],
       [ 120.        ,   14.16666667],
       [ 140.        ,   14.28571429],
       [ 160.        ,   15.        ],
       [ 180.        ,   15.55555556],
       [ 200.        ,   15.        ],
       [ 220.        ,   15.45454545],
       [ 240.        ,   16.25      ],
       [ 260.        ,   17.69230769],
       [ 280.        ,   18.57142857],
       [ 300.        ,   20.33333333],
       [ 320.        ,   20.9375    ],
       [ 340.        ,   21.17647059],
       [ 360.        ,   22.22222222],
       [ 380.        ,   22.36842105],
       [ 400.        ,   23.25      ],
       [ 420.        ,   23.57142857],
       [ 440.        ,   24.09090909],
       [ 460.        ,   25.43478261],
       [ 480.        ,   25.625     ],
       [ 500.        ,   26.        ],
       [ 520.        ,   